# Criação do Dataframe

In [ ]:
#pip install pandas
##pip install duckdb

In [4]:
import pandas as pd

class DataframeCreator:
  def __init__(self, csv_path):
    self.csv_path = csv_path

  def create_dataframe(self):
    #Lendo um arquivo csv com pandas
    try:
      dataframe = pd.read_csv(self.csv_path, encoding='latin-1', sep=',')
      return dataframe

    except FileNotFoundError:
      print(f"O arquivo não foi encontrado: {self.csv_path}")
      return None

    except pd.errors.EmptyDataError:
      print(f"O arquico está vazio: {self.csv_path}")
      return None

    except pd.errors.ParserError as e:
      print(f"Erro ao analisar o arquivo: {e}")
      return None

In [5]:

#Criar dataframe

csv_path = f'https://drive.google.com/uc?export=download&id=1p34yZ2KdgClf70KhopAxE5mA-7pArvFb'

dataframe_creator = DataframeCreator(csv_path)
df = dataframe_creator.create_dataframe()

if df is not None:
  print("Dataframe criado com sucesso")
  print(df.head())
  print(df.shape)

Dataframe criado com sucesso
  estado   ano CIRCOBITO   DTOBITO    DTNASC       SEXO RACACOR  \
0     AC  2014  Suicídio  02-01-14  02-07-77  Masculino   Preta   
1     AC  2014  Suicídio  23-01-14  30-07-66  Masculino  Branca   
2     AC  2014  Suicídio  31-01-14  28-07-43  Masculino  Branca   
3     AC  2014  Suicídio  05-02-14  04-07-99  Masculino  Branca   
4     AC  2014  Suicídio  06-02-14  02-05-72  Masculino  Branca   

             ESTCIV         ESC                             OCUP  \
0            Casado  4 a 7 anos   ATLETA PROFISSIONAL DE FUTEBOL   
1  União consensual   12 e mais  MEDICO GINECOLOGISTA E OBSTETRA   
2               NaN         NaN                                0   
3          Solteiro  4 a 7 anos                        ESTUDANTE   
4               NaN         NaN                                0   

        CODMUNRES    LOCOCOR ASSISTMED CAUSABAS CAUSABAS_O  idade  mes  
0      Rio Branco  Domicílio       Não     X700       X700   37.0    1  
1      Rio Br

# Query SQL

In [6]:
import duckdb

class SQLExecutor:
    def __init__(self, df):
        self.df = df

    def execute_sql(self, sql_script):
        # Criar uma conexão DuckDB usando o DataFrame
        con = duckdb.connect(database=':memory:',read_only=False)

        # Registrar o DataFrame como uma tabela DuckDB
        con.register('table_dataframe', self.df)

        # Executar o script SQL na tabela registrada
        result_dataframe = con.execute(sql_script).fetchdf()

        return result_dataframe

In [7]:
sql_script = """
SELECT
    ano as "Ano",
    estado as "Estado",
    sexo as "Sexo",
    ocup as "Ocupação",
    assistmed as "Assistência Médica",
FROM
    table_dataframe
WHERE
    ano like 2018
"""

sql_executor = SQLExecutor(df)
result_query = sql_executor.execute_sql(sql_script)

if result_query is not None:
  print(result_query.head())
  print(result_query.shape)

    Ano Estado       Sexo                            Ocupação  \
0  2018     AC  Masculino  TRABALHADOR VOLANTE DA AGRICULTURA   
1  2018     AC  Masculino               DESEMPREGADO CRONICO    
2  2018     AC  Masculino                                   0   
3  2018     AC  Masculino  TRABALHADOR VOLANTE DA AGRICULTURA   
4  2018     AC   Feminino                                   0   

  Assistência Médica  
0                Não  
1               None  
2               None  
3               None  
4                Não  
(12764, 5)


# Escrita em Parquet

In [8]:
class ParquetDataProcessor:
  def __init__(self, dataframe):
    self.dataframe = dataframe

  def save_to_parquet(self, output_file_path):
    try:
      self.dataframe.to_parquet(output_file_path, index=False)
      print(f"O Dataframe foi salvo em parquet: {output_file_path}")

    except Exception as e:
      print(f"Erro ao salvar o Dataframe em parquet: {e}")

In [9]:
processor = ParquetDataProcessor(df)

processor.save_to_parquet('datasus_suicidio_2014_2018.parquet')

O Dataframe foi salvo em parquet: datasus_suicidio_2014_2018.parquet
